In [1]:
import numpy as np
import pandas as pd     
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns    
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_squared_error

In [2]:
df = pd.read_csv("latimes-county-totals.csv")

In [3]:
df.head()

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
0,2020-01-26,Alameda,1,0,0,NaN,NaN
1,2020-01-31,Alameda,1,0,0,0.0,0.0
2,2020-02-02,Alameda,1,0,0,0.0,0.0
3,2020-02-20,Alameda,1,0,0,0.0,0.0
4,2020-02-21,Alameda,1,0,0,0.0,0.0


In [4]:
df.describe()

,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
count,2485.000000,2485.000000,2485.000000,2432.000000,2432.000000
mean,59.756539,229.698189,6.879276,15.485197,0.591283
std,32.786933,974.019813,37.795395,70.478349,3.502466
min,1.000000,0.000000,0.000000,-2.000000,0.000000
25%,31.000000,2.000000,0.000000,0.000000,0.000000
50%,65.000000,14.000000,0.000000,1.000000,0.000000
75%,85.000000,101.000000,2.000000,7.000000,0.000000
max,115.000000,16449.000000,732.000000,1474.000000,76.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 0 to 2484
Data columns (total 7 columns):
date                   2485 non-null object
county                 2485 non-null object
fips                   2485 non-null int64
confirmed_cases        2485 non-null int64
deaths                 2485 non-null int64
new_confirmed_cases    2432 non-null float64
new_deaths             2432 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 136.0+ KB


In [6]:
df_la = df[df['county'] == 'Los Angeles']

In [7]:
print(df_la)

           date       county  fips  confirmed_cases  deaths  \
657  2020-01-26  Los Angeles    37                1       0   
658  2020-01-27  Los Angeles    37                1       0   
659  2020-01-28  Los Angeles    37                1       0   
660  2020-01-29  Los Angeles    37                1       0   
661  2020-01-30  Los Angeles    37                1       0   
..          ...          ...   ...              ...     ...   
740  2020-04-18  Los Angeles    37            12051     577   
741  2020-04-19  Los Angeles    37            12349     601   
742  2020-04-20  Los Angeles    37            13823     619   
743  2020-04-21  Los Angeles    37            15165     666   
744  2020-04-22  Los Angeles    37            16449     732   

     new_confirmed_cases  new_deaths  
657                  NaN         NaN  
658                  0.0         0.0  
659                  0.0         0.0  
660                  0.0         0.0  
661                  0.0         0.0  
..       

In [8]:
days = list(range(1,len(df_la)+1))
len(days)

88

In [19]:
df_la['culm_day'] = days
#print(df_la)

df_la = df_la.loc[df_la.culm_day == df_la.date].copy()
print(df_la1)

     date       county  fips  confirmed_cases  deaths  new_confirmed_cases  \
657     1  Los Angeles    37                1       0                  NaN   
658     2  Los Angeles    37                1       0                  0.0   
659     3  Los Angeles    37                1       0                  0.0   
660     4  Los Angeles    37                1       0                  0.0   
661     5  Los Angeles    37                1       0                  0.0   
..    ...          ...   ...              ...     ...                  ...   
740    84  Los Angeles    37            12051     577                630.0   
741    85  Los Angeles    37            12349     601                298.0   
742    86  Los Angeles    37            13823     619               1474.0   
743    87  Los Angeles    37            15165     666               1342.0   
744    88  Los Angeles    37            16449     732               1284.0   

     new_deaths  culm_day  
657         NaN         1  
658    

In [20]:
df_la.columns

Index(['date', 'county', 'fips', 'confirmed_cases', 'deaths',
       'new_confirmed_cases', 'new_deaths', 'culm_day'],
      dtype='object')

In [21]:
df_la = df_la.drop(labels=['date','county','fips'], axis=1)
print (df_la.info())
df_la.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 657 to 744
Data columns (total 5 columns):
confirmed_cases        88 non-null int64
deaths                 88 non-null int64
new_confirmed_cases    87 non-null float64
new_deaths             87 non-null float64
culm_day               88 non-null int64
dtypes: float64(2), int64(3)
memory usage: 4.1 KB
None


,confirmed_cases,deaths,new_confirmed_cases,new_deaths,culm_day
count,88.000000,88.000000,87.000000,87.000000,88.000000
mean,2471.238636,85.931818,189.057471,8.413793,44.500000
std,4276.841374,175.960144,310.491559,15.787507,25.547342
min,1.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,0.000000,0.000000,22.750000
50%,19.500000,0.000000,5.000000,0.000000,44.500000
75%,3159.750000,57.000000,347.000000,11.000000,66.250000
max,16449.000000,732.000000,1474.000000,76.000000,88.000000


In [22]:
print(df_la)

     confirmed_cases  deaths  new_confirmed_cases  new_deaths  culm_day
657                1       0                  NaN         NaN         1
658                1       0                  0.0         0.0         2
659                1       0                  0.0         0.0         3
660                1       0                  0.0         0.0         4
661                1       0                  0.0         0.0         5
..               ...     ...                  ...         ...       ...
740            12051     577                630.0        76.0        84
741            12349     601                298.0        24.0        85
742            13823     619               1474.0        18.0        86
743            15165     666               1342.0        47.0        87
744            16449     732               1284.0        66.0        88

[88 rows x 5 columns]


In [23]:
df_la.dropna(inplace=True)

In [24]:
X = df_la.drop(labels=['confirmed_cases'], axis=1)
print (X)
y = df_la.loc[:,'confirmed_cases']
print (y)
print (y.unique())
X_col_names = X.columns.values
X_col_names

     deaths  new_confirmed_cases  new_deaths  culm_day
658       0                  0.0         0.0         2
659       0                  0.0         0.0         3
660       0                  0.0         0.0         4
661       0                  0.0         0.0         5
662       0                  0.0         0.0         6
..      ...                  ...         ...       ...
740     577                630.0        76.0        84
741     601                298.0        24.0        85
742     619               1474.0        18.0        86
743     666               1342.0        47.0        87
744     732               1284.0        66.0        88

[87 rows x 4 columns]
658        1
659        1
660        1
661        1
662        1
       ...  
740    12051
741    12349
742    13823
743    15165
744    16449
Name: confirmed_cases, Length: 87, dtype: int64
[    1     7    11    13    14    19    20    29    32    40    54    69
    94   147   192   231   295   353   411   543   67

array(['deaths', 'new_confirmed_cases', 'new_deaths', 'culm_day'],
      dtype=object)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(X_train)
print(len(X_train),len(y_train))
print(len(X_test),len(y_test))

     deaths  new_confirmed_cases  new_deaths  culm_day
718      26                252.0         5.0        62
707       1                 25.0         0.0        51
691       0                  0.0         0.0        35
697       0                  2.0         0.0        41
710       2                 39.0         1.0        54
681       0                  0.0         0.0        25
731     225                411.0        25.0        75
723      66                491.0        12.0        67
717      21                416.0         8.0        61
673       0                  0.0         0.0        17
728     147                436.0        15.0        72
736     366                595.0        40.0        80
684       0                  0.0         0.0        28
743     666               1342.0        47.0        87
712       4                 58.0         2.0        56
662       0                  0.0         0.0         6
704       1                  8.0         0.0        48
679       

In [26]:
print (X_test)
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax 

     deaths  new_confirmed_cases  new_deaths  culm_day
696       0                  4.0         0.0        40
702       1                  9.0         1.0        46
685       0                  0.0         0.0        29
698       0                  1.0         0.0        42
694       0                  0.0         0.0        38
740     577                630.0        76.0        84
716      13                144.0         3.0        60
727     132                630.0        13.0        71
744     732               1284.0        66.0        88
714       8                132.0         3.0        58
725      93                534.0        13.0        69
711       2                 64.0         0.0        55
706       1                 15.0         0.0        50
724      80                543.0        14.0        68
721      44                358.0         7.0        65
668       0                  0.0         0.0        12
660       0                  0.0         0.0         4
693       

array([[0.00000000e+00, 2.71370421e-03, 0.00000000e+00, 4.47058824e-01],
       [1.50150150e-03, 6.10583446e-03, 1.92307692e-02, 5.17647059e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.17647059e-01],
       [0.00000000e+00, 6.78426052e-04, 0.00000000e+00, 4.70588235e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.23529412e-01],
       [8.66366366e-01, 4.27408412e-01, 1.46153846e+00, 9.64705882e-01],
       [1.95195195e-02, 9.76933514e-02, 5.76923077e-02, 6.82352941e-01],
       [1.98198198e-01, 4.27408412e-01, 2.50000000e-01, 8.11764706e-01],
       [1.09909910e+00, 8.71099050e-01, 1.26923077e+00, 1.01176471e+00],
       [1.20120120e-02, 8.95522388e-02, 5.76923077e-02, 6.58823529e-01],
       [1.39639640e-01, 3.62279512e-01, 2.50000000e-01, 7.88235294e-01],
       [3.00300300e-03, 4.34192673e-02, 0.00000000e+00, 6.23529412e-01],
       [1.50150150e-03, 1.01763908e-02, 0.00000000e+00, 5.64705882e-01],
       [1.20120120e-01, 3.68385346e-01, 2.69230769e

## RBF


In [27]:
svr_rbf = SVR(kernel='rbf', C= 100, gamma = 0.1)

In [28]:
svr_rbf.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [29]:
y_predict = svr_rbf.predict(X_test)
y_predict

array([ 65.86995615, 146.08712254,   1.68867315,  34.06286747,
       133.53729164, 215.37636477, 215.37636477, 215.37636477,
       215.37636477, 219.88128501, 215.37636477, 216.75823472,
       132.99990972, 215.37636477, 215.37636477,   1.17664562,
        -1.73540485, 120.85904187, 215.37636477, 173.5633571 ,
         1.10369895, 215.37636477,  93.76589166,  35.67270888,
        16.30333359, 117.68424971, 215.42640992])

In [30]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

4837.736310828012

# Linear

In [31]:
svr_linear = SVR(kernel='linear', C= 100, gamma = 'auto')

In [32]:
svr_linear.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [33]:
y_predict = svr_linear.predict(X_test)
y_predict

array([ 1.61619052e+01,  1.72907809e+02,  5.53195564e+00,  1.15810976e+01,
        8.50999380e+00,  1.93868274e+04,  8.48505052e+02,  4.72039835e+03,
        2.14303444e+04,  7.55371772e+02,  3.99426484e+03,  1.54577627e+02,
        5.29939044e+01,  3.95548148e+03,  2.19432881e+03, -9.32275559e-02,
       -2.74037259e+00,  8.17910067e+00,  6.40383701e+03,  3.10308494e+01,
        2.88481061e+00,  1.39028109e+04,  7.84820754e+00,  7.18642128e+00,
        6.85552815e+00,  1.25737770e+01,  1.77977261e+03])

In [34]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

1807.1946529743277

# Polynomial


In [35]:
svr_poly = SVR(kernel='poly', C= 100, gamma = 'auto',degree = 2)

In [36]:
svr_poly.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [37]:
y_predict = svr_poly.predict(X_test)
y_predict


array([ 4.45145815e+01,  1.13272874e+03, -1.28658617e+01, -1.39564601e+01,
       -2.70555934e+01,  1.57154050e+05,  1.22350829e+03, -3.13868971e+04,
        1.80634314e+05,  2.69493409e+03, -1.90439934e+04,  1.56837180e+02,
        2.56394394e+01, -1.60991202e+04, -1.01808187e+04,  3.53586787e+00,
        6.54795021e+00, -2.52907014e+01,  1.47839992e+04, -2.09522083e+02,
       -3.45310442e+00, -1.11349720e+04, -2.35728732e+01, -2.02784081e+01,
       -1.87017713e+01, -1.86051498e+01, -1.11599613e+04])

In [38]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

43595.18784516646